In [1]:
import os
import sys
sys.path.append('utils/')
import loading_text_and_tokenization
import torch
import numpy as np
import torch.nn as nn
import random
import math

import utils.ngram_utils as ngram_utils
from utils.ngram_utils import NgramLM
from utils.amazon_dataset import AmazonDataset, pad, batchify
from torch.utils.data import DataLoader
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import utils.global_variables as gl
import torch
from tqdm import tqdm_notebook, tqdm
_tqdm = tqdm_notebook

In [2]:
torch.manual_seed(1)


In [3]:
use_cuda = True
device = torch.device("cuda" if (torch.cuda.is_available() and use_cuda) else "cpu")


In [4]:
# Read data from .txt files and create lists of reviews
train_data = []
# create a list of all the reviews 
with open('../data/small_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/small_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [5]:
train_data[0], valid_data[0]
train_data = train_data#[:100]
valid_data = valid_data#[:10]
train_data[0], type(train_data), len(train_data), type(train_data[0])

("this is a great tutu and at a really great price . it doesn ' t look cheap at all . i ' m so glad i looked on amazon and found such an affordable tutu that isn ' t made poorly . a + + ",
 list,
 2227,
 str)

In [6]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


In [7]:
N = 7

In [8]:
train_data_padded = ngram_utils.pad_dataset(train_data_tokenized, n=N)
valid_data_padded = ngram_utils.pad_dataset(valid_data_tokenized, n=N)

In [9]:
vocab = ngram_utils.get_vocab(train_data_padded)
vocab_size = len(vocab)
vocab_size, vocab[:10]

(6940, ('<sos>', '<eos>', '.', 'the', 'i', ',', 'and', 'a', 'to', "'"))

In [10]:
id2token, token2id = ngram_utils.get_dict(vocab)
len(id2token), len(token2id)

(6943, 6941)

In [11]:
train_data_ids = ngram_utils.get_ids(train_data_padded, token2id)
valid_data_ids = ngram_utils.get_ids(valid_data_padded, token2id)

In [12]:
train_dataset = AmazonDataset(train_data_ids, max_inp_length=None, use_cuda=True)
train_dataset_ngrams = []
for t in train_dataset:
    for i in range(len(t) - N):
        train_dataset_ngrams.append((t[i:i + N], t[i + N]))
train_loader = DataLoader(train_dataset_ngrams, batch_size=2048, collate_fn=batchify, shuffle=True)

100%|██████████| 10832/10832 [00:03<00:00, 3530.24it/s]


In [13]:
valid_dataset = AmazonDataset(valid_data_ids, max_inp_length=None, use_cuda=True)
valid_dataset_ngrams = []
for t in valid_dataset:
    for i in range(len(t) - N):
        valid_dataset_ngrams.append((t[i:i + N], t[i + N]))
valid_loader = DataLoader(valid_dataset_ngrams, batch_size=2048, collate_fn=batchify, shuffle=True)

100%|██████████| 1533/1533 [00:00<00:00, 41334.48it/s]


In [14]:
num_train = len(train_dataset_ngrams)
num_valid = len(valid_dataset_ngrams)
num_train, num_valid

(220156, 31775)

In [15]:
encoder = BagOfNGrams(len(id2token), emb_dim=300, hidden_size=256, out_size=128, activation='ReLU', nlayers=2, reduce='mean', dropout=0.1, batch_norm=False)
encoder

BagOfNGrams(
  (embedding): EmbeddingBag(6943, 300, mode=mean)
  (layers): ModuleList(
    (0): Linear(in_features=300, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1)
    (3): Linear(in_features=256, out_features=128, bias=True)
  )
)

In [16]:
decoder = DecoderMLP(input_size=128, output_size=len(id2token), hidden_size=256)
decoder

DecoderMLP(
  (linear): Linear(in_features=128, out_features=256, bias=True)
  (out): Linear(in_features=256, out_features=6943, bias=True)
  (log_softmax): LogSoftmax()
)

In [17]:
model = seq2seq(encoder, decoder, id2token, use_cuda=False, lr=0.1, size_ngrams=N) 
model

seq2seq(
  (encoder): BagOfNGrams(
    (embedding): EmbeddingBag(6943, 300, mode=mean)
    (layers): ModuleList(
      (0): Linear(in_features=300, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1)
      (3): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (decoder): DecoderMLP(
    (linear): Linear(in_features=128, out_features=256, bias=True)
    (out): Linear(in_features=256, out_features=6943, bias=True)
    (log_softmax): LogSoftmax()
  )
  (criterion): NLLLoss()
)

In [18]:
num_epochs = 10
log_interval = 10
best_eval_loss = np.inf

for epoch in range(num_epochs):
    # Train
    train_loss = 0   
    cur_loss = 0
    for i, (data, labels) in _tqdm(enumerate(train_loader)):
        prediction, loss = model.train_step(data, labels)
        train_loss += len(data) * loss
        cur_loss += loss
        
        if i % log_interval == 0 and i > 0:
            cur_loss = cur_loss / log_interval
            print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f} | {:5d}/{:5d} Batches'.format(
                epoch, cur_loss, math.exp(cur_loss), i, int(num_train/len(data))))
            cur_loss = 0
    
    train_loss = train_loss / num_train
    print('| Epoch {:3d} | Train Loss {:5.2f} | Train PPL {:8.2f}'.format(
            epoch, train_loss, math.exp(train_loss)))

    # Eval
    if epoch % 1 == 0:        
        eval_loss = 0
        for i, (data, labels) in _tqdm(enumerate(valid_loader)):
            prediction, loss = model.eval_step(data, labels)
            eval_loss += len(data) * loss
        eval_loss = eval_loss / num_valid 
        print('-' * 89)
        print('| Epoch {:3d} | Valid Loss {:5.2f} | Valid PPL {:8.2f}'.format(
            epoch, eval_loss, math.exp(eval_loss)))
        print('-' * 89)

        # Save the model if the validation loss is the best we've seen so far.
        if not best_eval_loss or eval_loss < best_eval_loss:
            with open('neural_lm_amazon_model_N{}'.format(N) + '.pt', 'wb') as f:
                torch.save(model, f)
            best_eval_loss = eval_loss

| Epoch   0 | Train Loss  8.26 | Train PPL  3849.74 |    10/  107 Batches
| Epoch   0 | Train Loss  6.39 | Train PPL   594.23 |    20/  107 Batches
| Epoch   0 | Train Loss  5.70 | Train PPL   300.25 |    30/  107 Batches
| Epoch   0 | Train Loss  5.10 | Train PPL   164.84 |    40/  107 Batches
| Epoch   0 | Train Loss  4.82 | Train PPL   123.54 |    50/  107 Batches
| Epoch   0 | Train Loss  4.67 | Train PPL   106.59 |    60/  107 Batches
| Epoch   0 | Train Loss  4.54 | Train PPL    93.27 |    70/  107 Batches
| Epoch   0 | Train Loss  4.49 | Train PPL    89.34 |    80/  107 Batches
| Epoch   0 | Train Loss  4.41 | Train PPL    82.25 |    90/  107 Batches
| Epoch   0 | Train Loss  4.38 | Train PPL    79.95 |   100/  107 Batches

| Epoch   0 | Train Loss  5.18 | Train PPL   177.09



-----------------------------------------------------------------------------------------
| Epoch   0 | Valid Loss  4.81 | Valid PPL   122.18
-----------------------------------------------------------------------------------------


| Epoch   1 | Train Loss  4.87 | Train PPL   130.15 |    10/  107 Batches
| Epoch   1 | Train Loss  4.34 | Train PPL    76.91 |    20/  107 Batches
| Epoch   1 | Train Loss  4.30 | Train PPL    74.01 |    30/  107 Batches
| Epoch   1 | Train Loss  4.41 | Train PPL    82.28 |    40/  107 Batches
| Epoch   1 | Train Loss  4.30 | Train PPL    73.93 |    50/  107 Batches
| Epoch   1 | Train Loss  4.36 | Train PPL    78.48 |    60/  107 Batches
| Epoch   1 | Train Loss  4.28 | Train PPL    72.12 |    70/  107 Batches


KeyboardInterrupt: 

In [ ]:
def score_sentence(sent):
    tokenized, _ = ngram_utils.tokenize_dataset(sent)
    sent_ids = ngram_utils.get_ids(tokenized, token2id)
    sent_tensor = torch.LongTensor(sent_ids).to(device)
    generated, scores = model.evaluate(sent_tensor, score_only=True)
    ppl = math.exp(scores)
    return ppl

In [ ]:
sent = ['this is a great tutu']
ppl = score_sentence(sent)
ppl

In [ ]:
sent = ['tutu tutu is not my favorit']
ppl = score_sentence(sent)
ppl

In [ ]:
sent = ['i really like this watch']
ppl = score_sentence(sent)
ppl

In [ ]:
sent = ['training neural networks']
ppl = score_sentence(sent)
ppl

In [ ]:
def generate_sentence(context=None):
    if context is None:
        dummy_context = torch.LongTensor([[0]]).to(device)
        generated, scores = model.evaluate(dummy_context, use_context=False)
    else:
        tokenized, _ = ngram_utils.tokenize_dataset(context)
        context_ids = ngram_utils.get_ids(tokenized, token2id)
        context_tensor = torch.LongTensor(context_ids).to(device)
        generated, scores = model.evaluate(context_tensor, use_context=True)
    
    ppl = math.exp(scores)
    return generated, scores

In [ ]:
generated, scores = generate_sentence()
print(' '.join(word[0] for word in generated))

In [ ]:
generated, scores = generate_sentence(context=['this is not my'])
print(' '.join(word[0] for word in generated))